# Quiz 7

In [33]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [34]:
W = np.random.normal(0, 1, (1000,))
X = W + np.random.normal(0, 1, (1000,))
Z = np.random.normal(0, 1, (1000,)) 
epsilon = np.random.normal(0, 1, (1000,))
Y = X + Z + W + epsilon

In [35]:
np.corrcoef(X, epsilon)[0, 1]

np.float64(0.016328358607082677)

In [38]:
bses = []
for n in range(1000):
    W = np.random.normal(0, 1, (1000,))
    X = W + np.random.normal(0, 1, (1000,))
    Z = np.random.normal(0, 1, (1000,)) 
    epsilon = np.random.normal(0, 1, (1000,))
    Y = X + Z + W + epsilon
    results = sm.OLS(Y, sm.add_constant(pd.DataFrame({'X': X, 'Z': Z, 'W': W}))).fit()
    bses.append(results.bse.iloc[1])

np.corrcoef(X, bses)[0, 1]

np.float64(0.013930025119425602)

In [39]:
W = np.random.normal(0, 1, (1000,))
X = W + np.random.normal(0, 1, (1000,))
Z = np.random.normal(0, 1, (1000,)) 
epsilon = W + np.random.normal(0, 1, (1000,))
Y = X + Z + epsilon

In [40]:
np.corrcoef(X, epsilon)[0, 1]

np.float64(0.531897739794845)

In [41]:
h71 = pd.read_csv("homework_7.1.csv", index_col=0)

In [42]:
h71

,X,W,Z,Y
0,1.137055,1.221768,0.327829,1.944532
1,-0.112905,0.465835,0.599650,0.655514
2,2.077755,1.795414,-0.063393,5.934411
3,0.456373,-0.512159,1.177413,-0.188064
4,-1.012402,0.080002,-0.275697,-0.533775
...,...,...,...,...
9995,2.569934,1.233620,0.930467,6.188783
9996,0.190150,1.022164,-0.015151,0.697187
9997,-1.184465,-1.475929,-0.287056,-1.575303
9998,-0.121286,-0.914357,1.706237,-1.809819


In [43]:
import numpy as np
import statsmodels.api as sm

targets = [-1, 0, 1]
starting_bandwidth = 0.2
min_rows = 50

for target_value in targets:
    bandwidth = starting_bandwidth

    # pick rows with W close to the target
    lower = target_value - bandwidth
    upper = target_value + bandwidth
    subset = h71[(h71["W"] >= lower) & (h71["W"] <= upper)]

    # widen the window if too few rows
    while len(subset) < min_rows and bandwidth < 0.8:
        bandwidth += 0.1
        lower = target_value - bandwidth
        upper = target_value + bandwidth
        subset = h71[(h71["W"] >= lower) & (h71["W"] <= upper)]

    # run Y on X and Z within this slice
    model = sm.OLS(subset["Y"], sm.add_constant(subset[["X", "Z"]])).fit()
    print("W near", target_value,
          "bandwidth", round(bandwidth, 2),
          "n", len(subset),
          "beta_X", float(model.params["X"]))


W near -1 bandwidth 0.2 n 967 beta_X 0.8699370837921349
W near 0 bandwidth 0.2 n 1546 beta_X 1.4177428595930375
W near 1 bandwidth 0.2 n 909 beta_X 1.9343326364400224


In [44]:
h71 = pd.read_csv("homework_7.1.csv", index_col=0)
h71

,X,W,Z,Y
0,1.137055,1.221768,0.327829,1.944532
1,-0.112905,0.465835,0.599650,0.655514
2,2.077755,1.795414,-0.063393,5.934411
3,0.456373,-0.512159,1.177413,-0.188064
4,-1.012402,0.080002,-0.275697,-0.533775
...,...,...,...,...
9995,2.569934,1.233620,0.930467,6.188783
9996,0.190150,1.022164,-0.015151,0.697187
9997,-1.184465,-1.475929,-0.287056,-1.575303
9998,-0.121286,-0.914357,1.706237,-1.809819


In [45]:
import numpy as np

# Make AR(1)-like errors with the user's formula
def make_error(corr_const, num, rng):
    err = []
    prev = rng.normal(0, 1)
    for _ in range(num):
        prev = corr_const * prev + (1 - corr_const) * rng.normal(0, 1)
        err.append(prev)
    return np.array(err)

# OLS with intercept: y = a + b*x
def fit_ols_intercept_x(y, x):
    n = len(y)
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    sxx = np.sum((x - x_mean) ** 2)
    sxy = np.sum((x - x_mean) * (y - y_mean))
    beta_hat = sxy / sxx
    alpha_hat = y_mean - beta_hat * x_mean

    # Residuals and IID-based SE (what OLS usually reports)
    y_hat = alpha_hat + beta_hat * x
    resid = y - y_hat
    sigma2_hat = np.sum(resid ** 2) / (n - 2)  # 2 parameters: a and b
    se_beta = np.sqrt(sigma2_hat / sxx)
    return beta_hat, se_beta

# Run many trials for one corr_const
def run_trials(corr_const, n_obs, n_trials, beta_true, rng):
    beta_hats = []
    se_hats = []

    for _ in range(n_trials):
        # Independent error processes for X and Y
        err_x = make_error(corr_const, n_obs, rng)
        err_y = make_error(corr_const, n_obs, rng)

        # Treatment X and outcome Y (include intercept in the regression step)
        x = err_x
        y = beta_true * x + err_y

        b, se = fit_ols_intercept_x(y, x)
        beta_hats.append(b)
        se_hats.append(se)

    beta_hats = np.array(beta_hats)
    se_hats = np.array(se_hats)

    # (i) empirical SD of estimates, (ii) mean reported OLS SE, and their ratio
    sd_beta = np.std(beta_hats, ddof=1)
    mean_se = np.mean(se_hats)
    ratio = sd_beta / mean_se
    return sd_beta, mean_se, ratio

if __name__ == "__main__":
    rng = np.random.default_rng(42)

    n_obs = 500        # number of rows per trial
    n_trials = 1000    # number of trials; increase for smoother numbers
    beta_true = 1.0
    corr_list = [0.2, 0.5, 0.8]

    print("corr_const |   sd(beta_hat)   |   mean OLS SE    |   ratio (i)/(ii)")
    print("-------------------------------------------------------------------")
    for c in corr_list:
        sd_b, mean_se, ratio = run_trials(c, n_obs, n_trials, beta_true, rng)
        # Fixed-point formatting to avoid scientific notation
        print(f"{c:>10.1f} | {sd_b:>15.6f} | {mean_se:>15.6f} | {ratio:>15.6f}")


corr_const |   sd(beta_hat)   |   mean OLS SE    |   ratio (i)/(ii)
-------------------------------------------------------------------
       0.2 |        0.047110 |        0.044720 |        1.053436
       0.5 |        0.058020 |        0.044878 |        1.292834
       0.8 |        0.102188 |        0.044318 |        2.305776
